In [1]:
import os
import pandas as pd
from google.cloud import storage
import requests

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.environ.get('GOOGLE_CREDENTIALS')
BUCKET = os.environ.get("GCP_GCS_BUCKET", 'dezoomcamp_2024_storage_bucket_radiant-gateway-412001')

init_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download'

In [2]:
def download_dataset(url, target):
    r = requests.get(url)
    open(target, 'wb').write(r.content)
            
def web_to_gcs_cvs(year, service):
    for i in range(1, 13):
        month = f'0{i}'
        month = month[-2:]
        file_name = f'{service}_tripdata_{year}-{month}.csv.gz'
        
        request_url = f'{init_url}/{service}/{file_name}'
        download_dataset(request_url, file_name)
        upload_csv_to_gcs(BUCKET, f'{service}_{year}_csv/{file_name}', file_name)

def upload_csv_to_gcs(bucket_name, object_name, local_file):
    storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB
    
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)


In [3]:
web_to_gcs_cvs('2019', 'green')
web_to_gcs_cvs('2020', 'green')